## 모델 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers;
!pip install sentencepiece;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
    DataCollatorForSeq2Seq,
)
import pickle
from drive.MyDrive.WARNING_PRIVATE_FOLDER.OSSP2AntiGPT9.ai.tokenization_kobert import KoBertTokenizer
#from drive.MyDrive.WARNING_PRIVATE_FOLDER.OSSP2AntiGPT9.ai.lib import tokenization_kobert
import json
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [ ]:
class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [ ]:
%cd /content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai
# with open('/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/save_src_tk.pickle','rb') as f:
#     src_tokenizer = pickle.load(f)
# there's an error in this code when activating in local

/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai


In [ ]:
with open("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/kobert_tokenizer_add/added_tokens.json", 'rb') as f:
    dw_dict = json.load(f)
dict_swap = {v:k for k,v in dw_dict.items()}
ma=12290
mi=8002
for i in range(8002,12291):
    src_tokenizer.add_tokens(dict_swap[i])

In [ ]:
model = EncoderDecoderModel.from_pretrained('leadawon/ossp-v0_3')
model.eval()
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id

## 테스트데이터 불러오기

In [ ]:
def read_input(path):
    with open(path, 'rb') as f:
        test_dict = json.load(f)
    test_standard = []
    test_dialect = []
    for pairs in test_dict['utterance']:
        test_standard.append(pairs['standard_form'])
        test_dialect.append(pairs['dialect_form'])

    return test_standard ,test_dialect

In [ ]:
test_s ,test_d= read_input("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/test_data_big.json")

## 후처리

In [ ]:
import re

In [ ]:
def translate(text, num_beam=1):
    embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
    embeddings = {k: v for k, v in embeddings.items()}
    output = model.generate(**embeddings, num_beams=num_beam)[0, 1:-1]
    tmp = trg_tokenizer.decode(output.cpu())
    tmp = re.sub('<unk>', '', tmp).strip()
    return tmp

In [ ]:
print("size of full : ",len(test_s))

size of full :  12451


In [ ]:
for i in range(len(test_s)):
    sr_text = test_d[i] # dialect
    tg_text = translate(sr_text) # translate
    print("source_text : ",sr_text)
    print("result :      ",tg_text)
    print("ground_truth :",test_s[i])
    print()
    print()
    if i%50==0:
        print("iteration num : ",i)
    if i==100:
        break

source_text :  언니 만낭 반가워 아 오늘 제주 방언 에이 아이 데이터
result :       언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터
ground_truth : 언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터


iteration num :  0
source_text :  어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예
result :       어 명절 설 명절 추석 명절 이렇게 나누어서 해볼 거예요
ground_truth : 어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예요


source_text :  자 그믄 이제부터 얘기해 보게예
result :       자 그러면 이제부터 얘기해 봐요
ground_truth : 자 그러면 이제부터 얘기해 봐요


source_text :  어 그믄 언니네 설 명절 때 음식 어떵 해
result :       어 그러면 언니네 설 명절 때 음식 어떻게 해
ground_truth : 어 그러면 언니네 설 명절 때 음식 어떻게 해


source_text :  음식을 막 준비했던 기억이 나게
result :       음식을 막 준비했던 기억이 나겠어요
ground_truth : 음식을 막 준비했던 기억이 나


source_text :  밀가룬지 쌀가룬지를 모르크라  근데 그거를 푹
result :       밀가루인지 쌀가루인지 모르겠어 근데 그거를 푹
ground_truth : 밀가룬지 쌀가룬지를 모르겠어  근데 그거를 푹


source_text :  반죽을 해그내 날 납작하게
result :       반죽을 해서 날 납작하게
ground_truth : 반죽을 해서 날 납작하게


source_text :  게 밀어 밀대로 밀어그내
result :       게 밀어 밀대로 밀어서
ground_truth : 게 밀어 밀대로 밀어서


source_text :  어 방바닥에다 똣똣한 방
result :       어 방바닥에다 따뜻한 방
ground_trut

AssertionError: ignored

## scoring

In [ ]:
!pip install "sacrebleu[ko]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sacrebleu.metrics import BLEU

In [ ]:
def score_B(sr_text, tg_text):

    candidate=[tg_text]
    reference=[[sr_text]]

    bleu = BLEU()

    bleuscore = bleu.corpus_score(candidate,reference).score
    return bleuscore
    #bleu_save.append(bleuscore)

In [ ]:
def bstest(tn, nb=1):
    sum_score = 0
    TEST_NUM = tn
    for i in range(TEST_NUM):
        sr_text = test_d[i] # dialect
        tg_text = translate(sr_text,nb) # translate
    #print("source_text : ",sr_text)
    #print("result      : ",tg_text)
    #print("ground_truth: ",test_s[i])
        scb = score_B(tg_text.strip(),test_s[i].strip())
    #print("bleu_socre  :  {:.2f}".format(scb))
    #print()
    #print()
        sum_score += scb
        if i%100==0:
            print("iteration num : ",i)
    print("bleu score mean : {}".format(sum_score/TEST_NUM))

In [ ]:
bstest(500,4) #60.306662232015825
bstest(500,3) #60.26833177503664
bstest(500,2) #60.40712645345418
bstest(500,1) #60.201772530857795

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


bleu score mean : 60.306662232015825
bleu score mean : 60.26833177503664
bleu score mean : 60.40712645345418
bleu score mean : 60.201772530857795


In [ ]:
# sum_score = 0
# TEST_NUM = 500
# for i in range(TEST_NUM):
#     sr_text = test_d[i] # dialect
#     tg_text = translate(sr_text,4) # translate
#     #print("source_text : ",sr_text)
#     #print("result      : ",tg_text)
#     #print("ground_truth: ",test_s[i])
#     scb = score_B(tg_text.strip(),test_s[i].strip())
#     #print("bleu_socre  :  {:.2f}".format(scb))
#     #print()
#     #print()
#     sum_score += scb
#     if i%50==0:
#         print("iteration num : ",i)
# print(sum_score/TEST_NUM)

## chatgpt api test

In [ ]:
!pip install openai

In [ ]:
import openai
import json
import time

In [ ]:
with open("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/chatgptapikey.json", 'rb') as f:
    tempjson = json.load(f)
    YOUR_API_KEY = tempjson["api_key"]

In [ ]:
def ChatGPT(prompt, API_KEY=YOUR_API_KEY):
    # api key 세팅
    openai.api_key = API_KEY

    # ChatGPT API 호출 및 최신 언어 모델인 text-davinci-003을 가져옴
    completion = openai.Completion.create(
        engine='text-davinci-003'  # 'text-curie-001'  # 'text-babbage-001' #'text-ada-001' # 'gpt-3.5-turbo'
        , prompt=prompt
        , temperature=0.5
        , max_tokens=64
        , top_p=1
        , frequency_penalty=0
        , presence_penalty=0
)

    return completion['choices'][0]['text']

In [ ]:
ChatGPT("이거 공짜로 해주면 안될까? ㅠㅠ")

In [ ]:
out_json = {"tp":[("제주방언","방언chatgpt","번역모델","번역모델chatgpt","정답")]}

In [ ]:
prompt_helper = "이 말을 이해했으면 1 아니면 0을 말해줘"
#prompt_helper = ""

In [ ]:
for i in range(len(test_s)):
    sr_text = test_d[i] # dialect
    tg_text = translate(sr_text) # translate
    print("source_text : ",sr_text)
    di_gpt = ChatGPT(sr_text+prompt_helper)
    print("chatgpt res : ",di_gpt)
    print("result :      ",tg_text)
    st_gpt = ChatGPT(tg_text+prompt_helper)
    print("chatgpt res : ",st_gpt)
    print("ground_truth :",test_s[i])
    print()
    print()
    out_json["tp"].append((sr_text,di_gpt,tg_text,st_gpt,test_s[i]))
    if i%50==0:
        print("iteration num : ",i)
    if i==200:
        print("end")
        break


In [ ]:
with open("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/chatgptapitest_result.json", "w") as outfile:
    json.dump(out_json, outfile,ensure_ascii=False)

In [ ]:
for i,v in enumerate(out_json["tp"]):
    out_json["tp"][i] = (v[0].strip(),v[1].strip(),v[2].strip(),v[3].strip(),v[4].strip())